In [1]:
# 텐서플로의 Gradient 테이프
import tensorflow as tf
x = tf.Variable(0.)
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [2]:
x = tf.Variable(tf.zeros(2,2))
with tf.GradientTape() as tape:
    y = 2*x+3
grad_of_y_wrt_x = tape.gradient(x,y) 

In [3]:
W = tf.Variable(tf.random.uniform((2,2))) # 가중치를 초기화
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
    y = tf.matmul(x,W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y,[W,b])

In [4]:
# 데이터 로드
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['acc'])

In [7]:
model.fit(x_train,y_train, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.2716 - acc: 0.9239
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1108 - acc: 0.9670
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0717 - acc: 0.9792
Epoch 4/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0504 - acc: 0.9853
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0365 - acc: 0.9894


In [8]:
# 텐서플로우를 이용해서 위의 식을 구현해보기

import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

In [9]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [10]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [11]:
#배치 제너레이터
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [12]:
# 훈련스텝 실행
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [13]:
# 가중치 업데이트
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [14]:
# 옵티마이저
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [15]:
# 훈련
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"에포크 {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

In [16]:
fit(model, x_train,y_train, epochs=10, batch_size=128)

에포크 0
0번째 배치 손실: 4.79
100번째 배치 손실: 2.22
200번째 배치 손실: 2.22
300번째 배치 손실: 2.06
400번째 배치 손실: 2.26
에포크 1
0번째 배치 손실: 1.90
100번째 배치 손실: 1.87
200번째 배치 손실: 1.84
300번째 배치 손실: 1.68
400번째 배치 손실: 1.85
에포크 2
0번째 배치 손실: 1.58
100번째 배치 손실: 1.57
200번째 배치 손실: 1.51
300번째 배치 손실: 1.39
400번째 배치 손실: 1.52
에포크 3
0번째 배치 손실: 1.32
100번째 배치 손실: 1.33
200번째 배치 손실: 1.24
300번째 배치 손실: 1.18
400번째 배치 손실: 1.28
에포크 4
0번째 배치 손실: 1.12
100번째 배치 손실: 1.15
200번째 배치 손실: 1.04
300번째 배치 손실: 1.02
400번째 배치 손실: 1.11
에포크 5
0번째 배치 손실: 0.97
100번째 배치 손실: 1.01
200번째 배치 손실: 0.90
300번째 배치 손실: 0.90
400번째 배치 손실: 0.99
에포크 6
0번째 배치 손실: 0.87
100번째 배치 손실: 0.90
200번째 배치 손실: 0.80
300번째 배치 손실: 0.81
400번째 배치 손실: 0.90
에포크 7
0번째 배치 손실: 0.78
100번째 배치 손실: 0.82
200번째 배치 손실: 0.72
300번째 배치 손실: 0.74
400번째 배치 손실: 0.83
에포크 8
0번째 배치 손실: 0.72
100번째 배치 손실: 0.75
200번째 배치 손실: 0.66
300번째 배치 손실: 0.69
400번째 배치 손실: 0.78
에포크 9
0번째 배치 손실: 0.67
100번째 배치 손실: 0.70
200번째 배치 손실: 0.61
300번째 배치 손실: 0.65
400번째 배치 손실: 0.74


In [17]:
# 모델평가하기
import numpy as np
predictions = model(x_test)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions,axis=1)
matches = predicted_labels == y_test
print(f"정확도:{matches.mean():.2f}")

정확도:0.82
